MIT License

Copyright (c) 2024 Justin Randall, Smart Interactive Transformations Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Overview

The purpose of this notebook is to serve as basic documentation for using the Interactigen library to generate base and augmented utterance phrases to use with natural language understanding (NLU) intent classification, or with Large Language Model (LLM) fine-tuning.

The Interactigen module is built on LangChain.  The decision to use LangChain enables experimenting with various models with minimal code adjustments.  Already have code to load and configure your favourite LangChain ChatModel?  Great!  Let's use that.

The output results for each run is a JSON array for downstream use: write to a file, split test/train utterances, and more.

## Generate Phrase Utterances with OpenAI

Here is an example loading the OpenAI gpt-4-0125-preview model for generating the utterances.

Make sure you have an environment file with the path *scratch/dev.env* and set the OPENAI_API_KEY and (optionally) LANGCHAIN_API_KEY and LANGCHAIN_PROJECT if you are tracing with LangSmith:
```scratch/dev.env:
OPENAI_API_KEY=<OpenAI API Key>
LANGCHAIN_API_KEY=<LangSmith API Key>
LANGCHAIN_PROJECT=<LangSmith Project Name>
```

In [1]:
#! pip install dotenv langchain_openai
from interactigen import Interactigen
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path='../scratch/dev.env')

model = ChatOpenAI(model="gpt-4-0125-preview")

interactigen = Interactigen(
    model=model,
)
 
phrase_utterances = interactigen.generate_phrase_utterances(
    base_phrase="someone might express appreciation to a bot",
    init_quantity=20,
)

phrase_utterances

['Thank you so much for your help!',
 'I really appreciate your assistance.',
 "You've been incredibly helpful, thanks!",
 'Thanks a lot for the support!',
 'Much obliged for your help.',
 "I'm so grateful for your assistance.",
 'Your support means a lot, thank you.',
 'Cheers for the help!',
 "You're a lifesaver, thanks!",
 "I can't thank you enough for your help.",
 'Thanks a bunch!',
 'Really appreciate it.',
 'Super helpful, thanks!',
 'Thanks for the support!',
 'Much obliged!',
 'So grateful, thanks!',
 'Support means a lot, thanks!',
 'Cheers!',
 "You're a lifesaver!",
 "Can't thank you enough!",
 'Thanks!',
 'Appreciated.',
 'So helpful!',
 'Thanks a lot!',
 'Obliged.',
 'Grateful.',
 'Supportive, thanks!',
 'Cheers!',
 'Lifesaver!',
 "Can't thank enough!",
 'Thank you so much for your help!',
 'I really appreciate your assistance.',
 "You've been incredibly helpful, thanks!",
 'Thanks a lot for the support!',
 'Much obliged for your help.',
 "I'm so grateful for your assistan

## Generate Phrase Utterances with Google Generative AI

Here is an example loading the Google Generative AI gemini-pro model for generating the utterances.

Make sure you have an environment file with the path *scratch/dev.env* and set the GOOGLE_API_KEY and (optionally) LANGCHAIN_API_KEY and LANGCHAIN_PROJECT if you are tracing with LangSmith:
```scratch/dev.env:
GOOGLE_API_KEY=<Google Generative AI API Key>
LANGCHAIN_API_KEY=<LangSmith API Key>
LANGCHAIN_PROJECT=<LangSmith Project Name>
```

In [3]:
#! pip install dotenv langchain_google_genai
from interactigen import Interactigen
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv(dotenv_path='../scratch/dev.env')

model = ChatGoogleGenerativeAI(model="gemini-pro")

interactigen = Interactigen(
    model=model,
)

phrase_utterances = interactigen.generate_phrase_utterances(
    base_phrase="someone might express appreciation to a bot",
    init_quantity=20,
)

phrase_utterances

['Thank you so much for your help!',
 'I really appreciate your assistance.',
 'Thanks a lot for the support!',
 "I'm grateful for your help.",
 'Much obliged for your assistance.',
 'Cheers for the guidance.',
 'Your support means a lot, thank you.',
 "I can't thank you enough for this.",
 'Your help has been invaluable.',
 'Thanks for everything!',
 "You've been a great help, thanks!",
 'Heartfelt thanks for your assistance.',
 "I'm so thankful for your help today.",
 'Your help is greatly appreciated.',
 'Many thanks for your assistance!',
 'I owe you one, thanks!',
 "You're a lifesaver, thank you.",
 'My sincere thanks for all your help.',
 'I deeply appreciate your help.',
 'Thank you kindly for your support.',
 'Thanks a bunch!',
 'Really appreciate it.',
 'Big thanks!',
 'So grateful for this.',
 'Thanks a ton.',
 'Cheers for the help.',
 'Means a lot, thanks.',
 "Can't thank you enough.",
 'Invaluable help, thanks.',
 'Thanks for everything!',
 'Thank you!',
 'Appreciated!',
 '